In [1]:
# jpg = 손실 압축 방식을 사용합니다. 이미지를 압축할 때 데이터 손실이 발생하며, 압축률이 높기 때문에 파일 크기를 작게 만들 수 있습니다. 그러나 이미지가 압축되면서 일부 세부 정보가 손실될 수 있습니다. 따라서 고해상도 사진에 적합하며, 색상이 많고 복잡한 이미지에 유리합니다.
# png = 무손실 압축 방식을 사용합니다. 이미지를 압축하는 동안 데이터 손실이 없으므로 원본 이미지를 완벽하게 보존합니다. 그러나 무손실 압축은 파일 크기가 상대적으로 크게 만들 수 있습니다. PNG는 그림, 아이콘, 투명한 배경이 필요한 이미지 등에 적합합니다.

# jpg = 파일 크기 작음, 압축 손실 발생
# png = 퍼알 크기 큼, 압축 손실 발생하지 않음

# vision 학습시 이미지 확장자가 jpg, png 중 무엇인지는 크게 영향이 없음

In [ ]:
# beautifulsoup 기본 명령어로 검색하면 원하는 기능 만들기 가능

In [2]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import sys
import os
import urllib.request as req
from PIL import Image
from glob import glob
import shutil
import pickle

In [3]:
target = "dia2"

In [4]:
# 크롤링할 홈페이지 주소와 게시판 마지막 번호

if target == "dia4":
    project_name = target
    base_url = "https://www.inven.co.kr/board/diablo4/6028"
    end_page_num = 162
    
elif target == "dia2":
    project_name = target
    base_url = "https://www.inven.co.kr/board/diablo2/5743"
    end_page_num = 1000

In [5]:
# 게시판 페이지마다 내부 게시글 주소 다 가져오기

notice_board_lst = []

page_range = [i+1 for i in range(0,end_page_num)]

for page_num in tqdm(page_range):
    target_url = base_url + f"?p={page_num}"
    
    response = requests.get(target_url) # url 정보 가져오기
    html_data = response.text # url 정보중 text만 추출

    soup = BeautifulSoup(html_data, 'html.parser') # html.parser로 나누기

    tags_with_classes = soup.find_all("a", class_ = 'subject-link') # html.parser에서 "a" 태그와 "subject_ling" 클래스에 있는 내용만 가져오기

    for tags_with_class in tags_with_classes: # tags_with_classes 에서 href 주소만 추출해서 list에 저장하기
        href_value = tags_with_class['href']
        notice_board_lst.append(href_value)
        
with open(f"{project_name}_board_url_lst.pickle", "wb")as f:
    pickle.dump(notice_board_lst, f)

100%|██████████| 1000/1000 [05:05<00:00,  3.27it/s]


In [ ]:
# 게시글 주소에서 이미지 url 다 가져오기

img_url_lst = []
with open(f"{project_name}_board_url_lst.pickle", "rb")as f:
    data = pickle.load(f)
for i in tqdm(data):
    response = requests.get(i) # url 정보 가져오기
    html_data = response.text # url 정보중 text만 추출
    
    soup = BeautifulSoup(html_data, 'html.parser') # html.parser로 나누기
    
    content_div = soup.find('div', id='powerbbsContent') # html.parser에서 id='powerbbsContent'인 <div>태그 선택
    
    img_tags = content_div.find_all('img') # 위 태그에서 img 부분만 추출
    
    for img_tag in img_tags:
        src_value = img_tag.get('src') # img src만 추출해서 list에 저장하기
        if src_value == None:
            continue
        img_url_lst.append(src_value)
        
with open(f"{project_name}_img_url_lst.pickle", "wb")as f:
    pickle.dump(img_url_lst, f)

 73%|███████▎  | 21904/30000 [2:14:42<32:34,  4.14it/s]  IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
# 이미지 url 저장하기

save_dir = os.path.join(os.getcwd(), f"{project_name}_imgs_src")
os.makedirs(save_dir, exist_ok=True)
cnt = 0

with open(f"{project_name}_img_url_lst.pickle", "rb")as f:
    data = pickle.load(f)
    
for img_url in tqdm(data):
    # path 설정
    ext_name = img_url.split("?MW=800")[0].split(".")[-1]
    save_img_base_name = str(cnt).zfill(12)
    save_img_name = save_img_base_name + "." + ext_name
    dst_path = os.path.join(save_dir, save_img_name)
    if os.path.isfile(dst_path):
        continue
    
    try:
        download_file = req.urlretrieve(img_url, dst_path) # url 저장
        cnt += 1
    except:
        continue
        
print(len(glob(os.path.join(save_dir,"*"))))

 44%|████▎     | 14888/34092 [1:54:10<2:33:00,  2.09it/s] IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 34092/34092 [2:32:16<00:00,  3.73it/s]

34086


In [8]:
# 저장 이미지 확장자 변경하기(list에 있는 것들에 대해 모두 실행)

# ext_lst = ["jpg", "png"]
ext_lst = ["jpg"]

imgs = glob(os.path.join(os.getcwd(), f"{project_name}_imgs_src", "*"))

for target_ext in ext_lst:
    dir_name = os.path.join(os.getcwd(), f"{project_name}_imgs_{target_ext}")
    os.makedirs(dir_name, exist_ok=True)

    for img in tqdm(imgs):
        tmp_lst = os.path.basename(img).split(".")
        base_name, ext_name = tmp_lst[0], tmp_lst[1]
        final_img_name = base_name + f".{target_ext}"
        dst_path = os.path.join(dir_name, final_img_name)

        if os.path.isfile(dst_path):
            continue

        if ext_name == target_ext:
            shutil.copy(img, dst_path)
            continue

        try:
            PIL_img = Image.open(img)
            PIL_img = PIL_img.convert("RGB")
            PIL_img.save(dst_path)
        except:
            continue

    print(len(glob(os.path.join(dir_name, "*"))))

100%|██████████| 34086/34086 [02:34<00:00, 221.05it/s]


34086
